# DatasetList test

In this notebook we test the functionalities of the `DatasetList` class.

## Labraries import

In [2]:
from caits.dataset._dataset3 import CaitsArray, DatasetList
from caits.loading import csv_loader
from caits.filtering import filter_butterworth

## Dataset loading

We load the data/GestureSet_small for this notebook.

In [3]:
data = csv_loader("data/GestureSet_small")
X, y, id = data["X"], data["y"], data["id"]
caitsX = [CaitsArray(values=x.values, axis_names={
    "axis_1": {
        col: i for i, col in enumerate(x.columns)
    }
}) for x in X]
type(caitsX[0]), type(y[0]), type(id[0])

Loading CSV files: 100%|██████████| 924/924 [00:00<00:00, 1273.43it/s]


(caits.dataset._dataset3.CaitsArray, str, str)

In [4]:
datasetListObj = DatasetList(caitsX, y, id)
datasetListObj

DatasetList object with 924 instances.

In [5]:
len(datasetListObj)

924

## Indexing

In this subsection we test the various indexing methods that can be used.

### Indexing using integer

This returns a `DatasetList` object, consisting of a single instance `X[int], y[int], _id[int]`.

In [6]:
datasetListObj[3]

DatasetList object with 1 instances.

### Indexing using a slice

This returns a `DatasetList` object, consisting of instances `X[slice], y[slice], _id[slice]`.

In [7]:
datasetListObj[3:15]

DatasetList object with 12 instances.

### Indexing using list of indices

This returns a `DatasetList` object, consisting of instances of the indices in the list `X[indices] y[indices], _id[indices]`.

In [8]:
datasetListObj[[3,8,16,107]]

DatasetList object with 4 instances.

### Indexing using a tuple of indices

This returns a `DatasetList` object, consisting of a single instance `X[int1][..., int2], y[int1], _id[int1]`.

In [11]:
datasetListObj[1, 4]

DatasetList object with 1 instances.

### Indexing using a tuple consisting of an integer and a slice

This returns a `DatasetList object`, consisting of a single instance `X[int][:, slice], y[int], _id[int]`.

In [12]:
tmp = datasetListObj[1, 2:5]
tmp, tmp.X[0].shape

(DatasetList object with 1 instances., (142, 3))

### Indexing using a tuple consisting of an integer and a list of integers

This returns a single `DatasetList` object, consisting o a single instance `X[int][:, list], y[int], _id[int]`.

In [13]:
tmp = datasetListObj[1, [3,4]]
tmp, tmp.X[0].shape

(DatasetList object with 1 instances., (142, 2))

### Indexing using column names

In this part, we will investigate indexing using column names.

In [14]:
datasetListObj.X[0].axis_names["axis_1"]

{'acc_x_axis_g': 0,
 'acc_y_axis_g': 1,
 'acc_z_axis_g': 2,
 'gyr_x_axis_deg/s': 3,
 'gyr_y_axis_deg/s': 4,
 'gyr_z_axis_deg/s': 5}

#### Indexing using a tuple, consisting of an integer and a column name

This will return a single `DatasetList` object, consisting of a single instance `X[int][..., col], y[int], _id[int]`.

In [ ]:
tmp = datasetListObj[1, "acc_x_axis_g"]
tmp, tmp.X[0].shape, tmp.X[0], tmp.y, tmp._id

#### Indexing using a tuple, consisting of an integer and a list of column names

This will return a single `DatasetList` object, constisting of a single instance `X[int][..., columns], y[int], _id[int]`.

In [ ]:
tmp = datasetListObj[1, ["acc_x_axis_g", "acc_z_axis_g"]]
tmp, tmp.X[0].shape

#### Indexing using a tuple, consising of an integer and a slice of column names.

This will return a single `DatasetList` object, consisting of a single instance `X[int][..., slice], y[int], _id[int]`.

In [ ]:
tmp = datasetListObj[1, "acc_x_axis_g":"gyr_x_axis_deg/s"]
tmp, tmp.X[0].shape, tmp.X[0]

### Indexing using tuple with first item a slice

#### Indexing using a tuple consisting of a slice and an integer

This will return a `DatasetList` object, consisting of multiple instances `X[slice][..., int], y[slice], _id[slice]`.

In [ ]:
datasetListObj[1:4, 1]

#### Indexing using a tuple consisting of two slices

This will return a `DatasetList` object, consisting of multiple instances `X[slice1][..., slice2], y[slice1], _id[slice1]`.

In [ ]:
datasetListObj[1:4, 3:5]

#### Indexing using a tuple consisting of a slice and a list of integers

This will return a `DatasetList` object, consisting of multiple instances `X[slice][..., list], y[slice], _id[slice]`.

In [ ]:
datasetListObj[1:4, [1,5]]

#### Indexing using a slice and a column name

This will return a `DatasetList` object, consisting of multiple instances `X[slice][..., col], y[slice], _id[slice]`.

In [ ]:
datasetListObj[1:4, "acc_x_axis_g"]

#### Indexing using a slice and a list of column names

This will return a `DatasetList` object, consisting of multiple instances `X[slice][..., list], y[slice], _id[slice]`.

In [ ]:
datasetListObj[1:4, ["acc_z_axis_g", "gyr_z_axis_deg/s"]]

#### Indexing using a slice of integers and a slice of column names

This will return a `DatasetList` object, consisting of multiple instances `X[slice1][..., slice2], y[slice1], _id[slice1]`.

In [ ]:
tmp = datasetListObj[1:4, "acc_x_axis_g":"gyr_x_axis_deg/s"]
tmp, tmp.X[0].shape, tmp.X[0]

In [ ]:
tmp1 = datasetListObj[:100, "acc_x_axis_g":"acc_z_axis_g"]
tmp2 = datasetListObj[:100, "gyr_x_axis_deg/s":"gyr_y_axis_deg/s"]
len(tmp1), len(tmp2), tmp1.X[0].shape, tmp2.X[0].shape

## Unify

In this subsection we test the unify. This method is used to merge `DatasetList` objects, row or column wise.

In [ ]:
axis_names = {**tmp1.X[0].axis_names["axis_1"], **tmp2.X[0].axis_names["axis_1"]}
axis_names

In [ ]:
tmp = tmp1.unify([tmp2], axis=1)
tmp, tmp.X[0].shape, tmp.X[0]

In [ ]:
tmp1 = datasetListObj[:100, ["acc_x_axis_g"]]
tmp2 = datasetListObj[:100, ["acc_y_axis_g"]]
tmp3 = datasetListObj[:100, ["acc_z_axis_g", "gyr_z_axis_deg/s"]]
tmp1.X[0], tmp2.X[0], tmp3.X[0]

In [ ]:
tmp = tmp1.unify([tmp2, tmp3], axis_names={"axis_1": {"col1": 0, "col2": 1, "col3": 2, "col4": 3}}, axis=1)
tmp, tmp.X[0].shape, tmp.X[0].axis_names

In [ ]:
tmp[:, ["col1", "col3"]].X

## Loops

In this subsection ww test looping capabilites of a `DatasetList` object.

### For loop

In [ ]:
for i, row in enumerate(datasetListObj):
    print(i)

### For loop in batches

In [ ]:
for i, batch in enumerate(datasetListObj.batch(10)):
    print(batch)

## Train_Test split

In this subsection we check the `train_test_split` method.

### Not-random split

This splits the `DatasetList` object in:
- train: first `Nx` instances
- test: last `N-Nx` instances

where `N` is the number of all instances and `Nx = int(N * (1 - test_size))`.

In [ ]:
train_obj, test_obj = datasetListObj.train_test_split()

In [ ]:
len(train_obj), len(test_obj)

In [ ]:
train_obj.X

### Random split

This splits the `DatasetList` object in:
- train: `Nx` random instances
- test: The rest `N-Nx` instances

where `N` is the number of all instances and `Nx = int(N * (1 - test_size))`.

In [ ]:
train_obj, test_obj = datasetListObj.train_test_split(random_state=42)
len(train_obj), len(test_obj)

## Adding two DatasetList objects

In this section we check the addition of two `DatasetList` objects. This is equivalent to:

`obj1.unify([obj2], axis=0)`

This way, the `obj2` is appended to the `obj1`, row-wise.

In [ ]:
newDatasetListObj = train_obj + test_obj
len(newDatasetListObj)

## Apply method

In this subsection we test applying a method on a `DatasetList` object.

When `DatasetList.apply` is called, the callable method is applied to the instances of `DatasetList.X`, one at a time.

In [ ]:
datasetListObj.apply(filter_butterworth, fs=200, filter_type='lowpass', cutoff_freq=50)